In [1]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
#from google.colab import files
from threading import Thread
from tqdm import *
#courses = pd.read_csv('courses_all.csv')
#career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])

In [3]:
eng_list = []

In [6]:
for i in tqdm(range(43)):
    page = rq.get('https://pharmacy.careers360.com/colleges/list-of-pharmacy-colleges-in-India?page='+str(i))
    soup = bs(page.content,'html.parser')
    vals = soup.find_all('div',class_='title')
    for val in vals:
        try:
            eng_list.append(val.a['href'])
        except:
            pass

100%|██████████| 43/43 [02:56<00:00,  4.10s/it]


In [2]:
courses = pd.read_csv('pharmacy_courses.csv')

In [9]:
courses = pd.DataFrame(columns=[0,1,2])

In [10]:
idx = 0
for link in tqdm(eng_list):
    try:
        page = rq.get(link)
        soup = bs(page.content,'html.parser')
        courses.loc[idx]=[link,soup.find_all('h4',class_='blockSubHeading')[0].get_text()[-2:],'https://engineering.careers360.com'+soup.find_all('ul',class_='list-tabs')[0].find_all('li')[1].a['href']]
        idx+=1
    except:
        pass

100%|██████████| 1270/1270 [45:43<00:00,  2.16s/it]


In [13]:
def subs(string):
    string2 = 'https://pharmacy.careers360.com/' + string[35:]
    return string2
courses[2] = courses[2].apply(subs)
courses.head()

,0,1,2
0,https://pharmacy.careers360.com/colleges/vello...,52,https://pharmacy.careers360.com/vellore-instit...
1,https://pharmacy.careers360.com/colleges/birla...,2,https://pharmacy.careers360.com/birla-institut...
2,https://pharmacy.careers360.com/colleges/guru-...,4,https://pharmacy.careers360.com/guru-gobind-si...
3,https://pharmacy.careers360.com/colleges/jadav...,4,https://pharmacy.careers360.com/colleges/jadav...
4,https://pharmacy.careers360.com/colleges/birla...,3,https://pharmacy.careers360.com/birla-institut...


In [8]:
def scrapper(start, end):
    career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])
    indx = 0
    cnt = start - 1
    for header_course in tqdm(courses['2'][start:end]):
        print(header_course)
        cnt += 1
        try:
            num = int(np.ceil(int(courses['1'][cnt])/10))
        except:
            pass
    #     print(f'num = {num}')
        for page_num in tqdm(range(num)):
            link = header_course + '?levelId=all&courseNid=all-all&page='+str(page_num)
    #         print(link)
            page = rq.get(link)
            course_soup = bs(page.content, 'html.parser')
        for course_link in course_soup.find_all('span',class_='readmore'):
            course_link = 'https://pharmacy.careers360.com/' + course_link.a['href']
            print(f'course_link = {course_link}')
            page1 = rq.get(course_link)
            soup = bs(page1.content,'html.parser')
            career360_courses.loc[indx]=None

            try:
                career360_courses.loc[indx]['Course'] = soup.find_all('h2',class_='block-title')[0].get_text()
                career360_courses.loc[indx]['College']=soup.find_all('h1',class_='titleNameCol')[0].get_text()
                career360_courses.loc[indx]['City'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[0].get_text()
                career360_courses.loc[indx]['State'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[1].get_text()
            except:
                pass

            try:
                career360_courses.loc[indx]['Eligiblity'] = soup.find_all('span',class_='more-eligibility')[0].get_text()[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Eligiblity'] = soup.find_all('div',class_='default-elig')[0].get_text()
                except:
                    career360_courses.loc[indx]['Eligiblity'] = None

            try:
                career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('span',class_='moreCourseDetails')[0].p.get_text())[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('div',class_='default-course-details')[0].get_text())
                except:
                    career360_courses.loc[indx]['Course_Detail'] = None

            for row in soup.find_all('div',class_='coursesPageLableInnerSec'):
                career360_courses.loc[indx][row.strong.get_text()] = row.p.get_text()
            indx+=1
    career360_courses.to_csv(f'career360_scrap_dm_pharmacy({start}-{end}).csv')

In [6]:
siz = (courses.shape[0])
numer = int(siz/20)
prev = 0
def threader():
    threads = []
    for i in range(19):
        thread = Thread(target=scrapper, args = (i*numer,(i+1)*numer))
        threads.append(thread)
        thread.start()
    thread = Thread(target=scrapper, args = (19*numer,siz))
    threads.append(thread)
    thread.start()                
    
    for thread in threads:  # iterates over the threads
        thread.join()
                    
    print('done')

In [24]:
def threader():
    t3 = Thread(target=scrapper, args = (0,100))
    t4 = Thread(target=scrapper, args = (100,200))
    t5 = Thread(target=scrapper, args = (200,300))
    t6 = Thread(target=scrapper, args = (300,400))
    t3.start()
    t4.start()
    t5.start()
    t6.start()
    t3.join()
    t4.join()
    t5.join()
    t6.join()
    print('done')

In [9]:
threader()

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/63 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/63 [00:00<?, ?it/s]






  0%|          | 0/1 [00:00<?, ?it/s]







  0%|          | 0/63 [00:00<?, ?it/s]








  0%|          | 0/1 [00:00<?, ?it/s]









  0%|          | 0/63 [00:00<?, ?it/s]










  0%|          | 0/1 [00:00<?, ?it/s]











  0%|          | 0/63 [00:00<?, ?it/s]












  0%|          | 0/1 [00:00<?, ?it/s]













  0%|          | 0/63 [00:00<?, ?it/s]














  0%|          | 0/1 [00:00<?, ?it/s]















  0%|          | 0/63 [00:00<?, ?it/s]
















  0%|          | 0/1 [00:00<?, ?it/s]

















  0%|          | 0/63 [00:00<?, ?it/s]


















  0%|          | 0/1 [00:00<?, ?it/s]





















https://pharmacy.careers360.com/vellore-institute-of-technology-vellore/branches
https://pharmacy.careers360.com/siksha-o-anusandhan-bhubaneswar/branches
https://pharmacy.careers360.com/colleges/tamil-nadu-dr-mgr-medical-university-chennai/courses
https://pharmacy.careers360.com/colleges/government-college-of-pharmacy-aurangabad/courses
https://pharmacy.careers360.com/dr-bc-roy-college-pharmacy-and-allied-health-sciences-durgapur/branches
https://pharmacy.careers360.com/colleges/bhupal-nobles-university-udaipur/courses
https://pharmacy.careers360.com/colleges/vinayaka-missions-sikkim-university-sikkim/courses
https://pharmacy.careers360.com/colleges/maharajahs-college-of-pharmacy-phool-baugh/courses
https://pharmacy.careers360.com/colleges/shree-santkrupa-college-of-pharmacy-karad/courses
https://pharmacy.careers360.com/colleges/maa-saraswati-college-of-pharmacy-abohar/courses


  0%|          | 0/63 [00:00<?, ?it/s]




















  0%|          | 0/1 [00:00<?, ?it/s]





















  0%|          | 0/63 [00:00<?, ?it/s]






















  0%|          | 0/1 [00:00<?, ?it/s]























  0%|          | 0/63 [00:00<?, ?it/s]
























  0%|          | 0/1 [00:00<?, ?it/s]

























  0%|          | 0/63 [00:00<?, ?it/s]


























  0%|          | 0/1 [00:00<?, ?it/s]



























  0%|          | 0/63 [00:00<?, ?it/s]




























  0%|          | 0/1 [00:00<?, ?it/s]





























  0%|          | 0/63 [00:00<?, ?it/s]






























  0%|          | 0/1 [00:00<?, ?it/s]































  0%|          | 0/71 [00:00<?, ?it/s]
































  0%|          | 0/1 [00:00<?, ?it/s]

































  0%|          | 0/63 [00:00<?, ?it/s]


































  0%|          | 

https://pharmacy.careers360.com/colleges/akal-college-of-pharmacy-and-technical-education-mastuana/courses
https://pharmacy.careers360.com/colleges/shriman-suresh-dada-jain-college-of-pharmacy-chandwad/courses
https://pharmacy.careers360.com/colleges/sri-venkateswara-college-of-pharmacy-rvs-nagar/courses
https://pharmacy.careers360.com/colleges/kjr-college-of-pharmacy-rajahmundry/courses
https://pharmacy.careers360.com/colleges/krupanidhi-college-of-pharmacy-bangalore/courses
https://pharmacy.careers360.com/colleges/auroras-pharmacy-college-bhongir/courses
https://pharmacy.careers360.com/colleges/pk-university-shivpuri/courses
https://pharmacy.careers360.com/colleges/hke-societys-matoshree-taradevi-rampure-institute-of-pharmaceutical-sciences-kalaburagi/courses
https://pharmacy.careers360.com/colleges/smt-vidyawati-college-of-pharmacy-jhansi/courses
https://pharmacy.careers360.com/colleges/shree-hn-shukla-institute-of-pharmaceutical-education-and-research-rajkot/courses



 17%|█▋        | 1/6 [00:01<00:08,  1.70s/it]


100%|██████████| 1/1 [00:01<00:00,  1.87s/it]








100%|██████████| 1/1 [00:01<00:00,  1.86s/it]
























100%|██████████| 1/1 [00:01<00:00,  1.70s/it]



































100%|██████████| 1/1 [00:01<00:00,  1.75s/it]























100%|██████████| 1/1 [00:01<00:00,  1.89s/it]






























100%|██████████| 1/1 [00:01<00:00,  1.68s/it]





































100%|██████████| 1/1 [00:01<00:00,  1.84s/it]























100%|██████████| 1/1 [00:01<00:00,  1.90s/it]

















100%|██████████| 1/1 [00:01<00:00,  1.95s/it]


































100%|██████████| 1/1 [00:01<00:00,  1.71s/it]






















































100%|██████████| 1/1 [00:01<00:00,  1.73s/it]





























...

https://pharmacy.careers360.com/colleges/sun-institute-of-pharmaceutical-education-and-research-nellore/courses




100%|██████████| 1/1 [00:01<00:00,  1.67s/it]



course_link = https://pharmacy.careers360.com//colleges/sun-institute-of-pharmaceutical-education-and-research-nellore/courses/b-pharma
course_link = https://pharmacy.careers360.com//colleges/sun-institute-of-pharmaceutical-education-and-research-nellore/courses/m-pharma-pharmaceutical-analysis
course_link = https://pharmacy.careers360.com//colleges/sun-institute-of-pharmaceutical-education-and-research-nellore/courses/m-pharma-pharmaceutics


100%|██████████| 63/63 [15:39<00:00, 14.92s/it]

done


In [31]:
courses.head()

,0,1,2
0,https://pharmacy.careers360.com/colleges/vello...,52,https://pharmacy.careers360.com/vellore-instit...
1,https://pharmacy.careers360.com/colleges/birla...,2,https://pharmacy.careers360.com/birla-institut...
2,https://pharmacy.careers360.com/colleges/guru-...,4,https://pharmacy.careers360.com/guru-gobind-si...
3,https://pharmacy.careers360.com/colleges/jadav...,4,https://pharmacy.careers360.com/colleges/jadav...
4,https://pharmacy.careers360.com/colleges/birla...,3,https://pharmacy.careers360.com/birla-institut...


In [41]:
scrapper(0,3)















  0%|          | 0/3 [00:00<?, ?it/s]














  0%|          | 0/6 [00:00<?, ?it/s]

https://pharmacy.careers360.com/vellore-institute-of-technology-vellore/branches

















 17%|█▋        | 1/6 [00:01<00:05,  1.16s/it]














 33%|███▎      | 2/6 [00:02<00:04,  1.15s/it]














 50%|█████     | 3/6 [00:03<00:03,  1.14s/it]














 67%|██████▋   | 4/6 [00:04<00:02,  1.14s/it]














 83%|████████▎ | 5/6 [00:04<00:00,  1.00it/s]














100%|██████████| 6/6 [00:06<00:00,  1.02s/it]




























 33%|███▎      | 1/3 [00:06<00:12,  6.13s/it]














  0%|          | 0/1 [00:00<?, ?it/s]

https://pharmacy.careers360.com/birla-institute-technology-and-science-pilani/courses-offered

















100%|██████████| 1/1 [00:00<00:00,  2.86it/s]
















...















 67%|██████▋   | 2/3 [00:12<00:06,  6.23s/it]














  0%|          | 0/1 [00:00<?, ?it/s]

https://pharmacy.careers360.com/guru-gobind-singh-indraprastha-university-faculty-engineering-delhi/branches

















100%|██████████| 1/1 [00:00<00:00,  2.71it/s]
















course_link = https://pharmacy.careers360.com//colleges/guru-gobind-singh-indraprastha-university-delhi/courses/bpharma
course_link = https://pharmacy.careers360.com//colleges/guru-gobind-singh-indraprastha-university-delhi/courses/bsc-mlt
course_link = https://pharmacy.careers360.com//node/6500/course/44956
course_link = https://pharmacy.careers360.com//colleges/guru-gobind-singh-indraprastha-university-delhi/courses/md-pathology
















100%|██████████| 3/3 [00:23<00:00,  7.96s/it]













